# 使用可变长参数进行前向传播和反向传播
> 拓展Fuction类，将传入函数实例的参数和结果设置为可变成长度的列表格式

In [2]:
#原
import numpy as np
class Variable:
    def __init__(self,data):
        if data:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self,func):
        self.creator = func
    def backward(self):
        f = self.creator
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

def is_array(input):
    if np.isscalar(input):
        return np.array(input)
    return input

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(is_array(y))
        output.set_creator(self)
        self.input  =input
        self.output = output
        return output
    def forward(self,input):
        raise NotImplementedError()
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        return x**2
    def backward(self,gy):
        return 2*self.input.data*gy
    
class Exp(Function):
    def forward(self,x):
        return np.exp(x)
    def backward(self,gy):
        return np.exp(self.input.data)*gy

def square(x):
    return Square()(x)
def exp(x):
    return np.Exp()(x)



In [11]:
#基本拓展
import numpy as np
class Variable:
    def __init__(self,data):
        if data:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self,func):
        self.creator = func
    def backward(self):
        f = self.creator
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

def is_array(input):
    if np.isscalar(input):
        return np.array(input)
    return input
#拓展一
class Function:
    def __call__(self,inputs):
        xs = [x.data for x in inputs]
        ys =self.forward(xs)
        outputs = [Variable(is_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    def forward(self,xs):
        raise NotImplementedError()
    def backward(self,gy):
        raise NotImplementedError()
#拓展二
class Add(Function):
    def forward(self,xs):
        x1 , x2 = xs
        y = x1 + x2
        return (y,) #返回值也需要是列表或元组
    

In [14]:

#测试
x0 = Variable(np.array(2.0))
x1 = Variable(np.array(3.0))
xs = [x0, x1] 
A = Add()
ys = A(xs)
y = ys.data
print(y)

5.0
